In [1]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("max_colwidth", None)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


def df_stats(df):
    from tabulate import tabulate

    while True:
        print("\n***** Shape: ", df.shape, " *****\n")

        columns_list = df.columns.values.tolist()
        isnull_list = df.isnull().sum().values.tolist()
        isunique_list = df.nunique().values.tolist()
        dtypes_list = df.dtypes.tolist()

        list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
        df_stat_val = pd.DataFrame(list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"])
        print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

/tmp/ipykernel_512906/441343652.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import HTML, display


In [ ]:
# Load the CSV file with alpha (α) separator

# Try loading with alpha separator
df = pd.read_csv(
    'data/reports/reports_with_alpha_separator_with_Calcifc_Stenosis_IFR_20250507_STUDYLEVEL.csv', 
    sep='α'
)



In [ ]:
display(df['SeriesInstanceUID'])

In [ ]:
import os
from tqdm import tqdm

# Create a new column with the embedding path
df['EmbeddingPath'] = '/media/data1/datasets/DeepCoro_CLIP/video_embeddings/xvwwv5ar/' + df['SeriesInstanceUID'] + '.pt'

# Check if the embedding files exist
def check_file_exists(path):    
    return os.path.exists(path)

# Apply the check to each row with a progress bar
print("Checking if embedding files exist...")
df['EmbeddingExists'] = list(tqdm(map(check_file_exists, df['EmbeddingPath']), total=len(df)))

# Display summary statistics
print(f"\nTotal files: {len(df)}")
print(f"Files that exist: {df['EmbeddingExists'].sum()}")
print(f"Files missing: {len(df) - df['EmbeddingExists'].sum()}")

# Display a sample of the dataframe with the new columns
display(df[['SOPInstanceUID', 'EmbeddingPath', 'EmbeddingExists']].head())


In [9]:
import torch
from models.video_encoder import VideoEncoder

# Load full checkpoint dict (not just the state dict)
checkpoint = torch.load("outputs/DeepCORO_clip/dev_deep_coro_clip_single_video/6lex8uok_20250531-104348/checkpoints/checkpoint.pt")

# Extract just the video encoder weights
state_dict = checkpoint["video_encoder"]  # make sure this key exists!

# Rebuild the model with the same architecture
model = VideoEncoder(
    backbone="mvit", 
    input_channels=3,
    num_frames=16,
    pretrained=False,              # We will load trained weights manually
    output_dim=512,
    dropout=0.2,
    num_heads=8,
    freeze_ratio=0.8,
    aggregator_depth=4,
    aggregate_videos_tokens=True,  # Can toggle this later for inference
    per_video_pool=False
)

# Load only the video_encoder weights
model.load_state_dict(state_dict)
model.eval().cuda()
# Prepare your input tensor
# Expected shape: [B, N, T, H, W, C]
# For example:
input_tensor = torch.randn(2, 1, 16, 224, 224, 3).cuda()

# To get patch-level tokens: [B, N * L, D]
patch_tokens = model.get_tokens(input_tensor, mode="patch")
display(patch_tokens.shape)
# To get video-level tokens: [B, N, D]
video_tokens = model.get_tokens(input_tensor, mode="video")
display(video_tokens.shape)

/tmp/ipykernel_512906/1404217126.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("outputs/DeepCORO_clip/dev_deep_coro_clip_single_video/6lex8uok_

torch.Size([2, 393, 512])

torch.Size([2, 1, 512])